In [ ]:
import pandas as pd
import joblib
from datetime import datetime

# تحميل النموذج من الملف
loaded_model = joblib.load('/kaggle/input/best-model-pkl/best_model.pkl')

# دالة لتحويل الإدخال النصي إلى صيغة مناسبة
def preprocess_input(date_str, place, vip_people, arena_status_str, weather_str):
    new_event = {
        'date': date_str,
        'place': place,
        'vip_people': vip_people,
        'arena_status': arena_status_str,
        'weather': weather_str
    }

    # تحويل البيانات إلى DataFrame
    new_event_df = pd.DataFrame([new_event])

    # تحويل القيم الفئوية
    new_event_df['date'] = pd.to_datetime(new_event_df['date'])
    new_event_df['arena_status'] = new_event_df['arena_status'].map({'مفتوحة': 1, 'مغلقة': 0})
    new_event_df['weather'] = new_event_df['weather'].map({'مشمس': 1, 'غائم': 2, 'ممطر': 3, 'عاصف': 4, 'بارد': 5, 'جاف': 6})

    # استخراج ميزات إضافية
    new_event_df['day_of_week'] = new_event_df['date'].dt.dayofweek
    new_event_df['month'] = new_event_df['date'].dt.month

    # تحديد الأعمدة المستخدمة في النموذج
    X_new = new_event_df[['day_of_week', 'month', 'arena_status', 'vip_people', 'weather']]

    # التحقق من شكل البيانات
    if X_new.shape[1] != 5:
        raise ValueError("البيانات المدخلة ليست بالشكل الصحيح. تأكد من إدخال جميع الأعمدة المطلوبة.")

    return X_new

# دالة لجعل المستخدم يدخل المعلومات المطلوبة
def get_user_input():
    place = input("أدخل اسم المكان: ")
    date_str = input("أدخل تاريخ الفعالية (YYYY-MM-DD): ")
    vip_people = int(input("أدخل عدد الشخصيات الهامة (VIP): "))
    arena_status_str = input("أدخل حالة القاعة (مفتوحة/مغلقة): ")
    weather_str = input("أدخل حالة الطقس (مشمس/غائم/ممطر/عاصف/بارد/جاف): ")

    return date_str, place, vip_people, arena_status_str, weather_str

# دالة لتنفيذ التنبؤ
def predict_attendance(date_str, place, vip_people, arena_status_str, weather_str):
    X_new = preprocess_input(date_str, place, vip_people, arena_status_str, weather_str)
    future_prediction = loaded_model.predict(X_new)
    return int(future_prediction[0])

# واجهة المستخدم الرئيسية
def main():
    print("مرحبا! أهلا بك في نظام التنبؤ بعدد الحضور.")
    date_str, place, vip_people, arena_status_str, weather_str = get_user_input()
    predicted_attendance = predict_attendance(date_str, place, vip_people, arena_status_str, weather_str)
    print(f'التنبؤ بعدد الحضور المتوقع: {predicted_attendance}')

# تنفيذ البرنامج
if __name__ == "__main__":
    main()
